# Importar los paquetes necesarios para trabajar

In [ ]:
# Pandas nos permitirá exportar los resultados, 
# Bio será el que nos permitirá leer las secuencias fasta,
# Os nos permitirá crear carpetas y renombrar archivos
# regex nos permite buscar secuencias que tengas mismatches
# shutil nos sirve para mover archivos de una carpeta a otra

import pandas as pd 
from pandas import DataFrame

import Bio
from Bio import SeqIO
from Bio.SeqUtils import GC

import os

import regex

import shutil


# Crear el espacio de trabajo en el que guardaremos nuestros resultados

In [ ]:
#modificaremos el path cada vez que vayamos a utilizarlo
path = "C:/Users/nerit/Desktop/mouse and rat genes/"
os.chdir(path)

folder = "kmers" #aquí se guardarán todos los kmers obtenidos con la primera función, sin filtrar de ninguna forma
resultsmouse = "Mouse"
resultsrat = "Rat"

os.makedirs(folder)
os.makedirs(resultsmouse)
os.makedirs(resultsrat)

path2 = "C:/Users/nerit/Desktop/mouse and rat genes/kmers"
path3 = "C:/Users/nerit/Desktop/mouse and rat genes/Mouse"
path4 = "C:/Users/nerit/Desktop/mouse and rat genes/Rat"

kmers = "kmers"
filteredkmers = "kmers_filtered" #guardar los kmers tras filtrar para que sean compatibles en todas las especies
fusedkmers = "kmers_fused" #guardar los kmers filtrados sabiendo su posición en el mRNA

os.chdir(path3)
os.makedirs(kmers)
os.makedirs(filteredkmers)
os.makedirs(fusedkmers)

os.chdir(path4)
os.makedirs(kmers)
os.makedirs(filteredkmers)
os.makedirs(fusedkmers)

# Definición de las funciones

Definiremos una primera función que busque los kmers posibles para cada secuencia. Además, los filtrará. 

In [ ]:
# function to do a sliding window and extracts all the 30-mers from the fasta sequence

def findtargets (mrna):
    targets = pd.DataFrame(columns=['Gene', 'Position', 'Sequence'])
    end = len(mrna)-29
    #print (end)
    for i in range (0, end):
        #print (mrna[i:i+30])
    #The next line checks if position 16 (remember python is 0-indexed) is a C or G
        if mrna.seq[i+15] == 'C' or mrna.seq[i+15] == 'G' :
            #The next line filters out any probe with GC content <= 50 and >=65
            if GC(mrna.seq[i:i+30]) > 50:
                if GC(mrna.seq[i:i+30]) < 65:
                    if mrna.seq[i:i+30].count("AAA")==0 and mrna.seq[i:i+30].count("TTT")==0 and mrna.seq[i:i+30].count("GGG")==0 and mrna.seq[i:i+30].count("CCC")==0:
                    #Here I create a dataframe with all the suitable targets, where column 1 is the start position and column 2 is the actual sequence.
                        #print (GC(mrna.seq[i:i+30]))
                        targets = targets.append({'Gene': mrna.id, 'Position': i, 'Sequence':mrna.seq[i:i+30]}, ignore_index=True)  
                        targets.to_csv(r'C:/Users/nerit/Desktop/mouse and rat genes/kmers/kmers_%s.csv' %mrna.description)

    return [targets]  

La segunda función sirve para renombrar los archivos .csv para que tengan sólo el nombre "común" del gen. Esta función solo sirve para poner el nombre que se encuentre entre paréntesis, así que asegurarse de que los archivos iniciales tengan el formato adecuado. El archivo que contendrá los kmers estará nombrado por toda la descripción del gen (así está especificado en la función findtargets), dentro de la cual el nombre "común" aparecerá así: (Adarb2). Será ese con el que nos quedemos. Por si acaso, es muy importante revisar los nombres de todos los archivos antes, para comprobar que no haya ninguna otra palabra entre paréntesis. 

In [ ]:
def change_name (gene):
    #primero el bucle, que permite leer cada archivo en nuestro directorio
    for f in os.listdir():
        #"rompe" el nombre, poniendo el texto (f_name) por un lado y la extensión del documento (f_ext) por otro
        f_name, f_ext = os.path.splitext(f)
        #print(f_name)
        
        #"rompe" el nombre en una lista: cada palabra/elemento del título será un item independiente
        f_gene = f_name.split()
        
        #crea un bucle para leer cada item de la lista correspondiente al nombre
        for element in f_gene:
            
            #crear la condición que hace que nos centremos sólo en el ítem que tiene paréntesis
            if element [0] == "(":
                #print(element)
                
                #una vez centrado en el item del "nombre común", lo modifica para quitar los paréntesis (con la función strip)
                #y para quitarle el espacio.
                element = element.strip()[1:]
                element = element.strip()[:-1]
                element = element.strip()[:-1]
                #print(element)
                
                #generar nuevo nombre para el archivo, poniendo seguidos el item del nombre común sin paréntesis y la extensión
                #renombrar
                new_name = '{}{}'.format(element, f_ext)
                #print(new_name)
                os.rename(f, new_name)

# Ejecutar la función para buscar kmers

Una vez tenemos todo definido, lo primero que vamos a ejecutar es la función que nos busque los kmers. Lo que hará será tomar todas las secuencias de todos los archivos y creará un excel con los resultados por archivo y secuencia. # Es muy importante que en el archivo estén las DOS secuencias (una correspondiente a una especie y otra correspondiente a la otra), porque sólo con una no funciona el script.

In [ ]:
#definir el path en el que están los archivos con las secuencias
filepath = "C:/Users/nerit/Desktop/mouse and rat genes/sequences/"
files = os.listdir(filepath)
#print (files)

#bucle para que lea y analice cada archivo de forma individual
for gene in files:
    
    #esta función nos ayuda a indicarle a python que las secuencias están en formato FASTA, para que las trate de esa manera
    seq_files = SeqIO.parse(filepath+gene, "fasta")
    print(seq_files)
    
    #bucle para que lea cada una de las dos secuencias de forma individual. 
    for seq in seq_files:
        #seqcount=seqcount+1
        #print ('secuencias procesadas')
        #print(seqcount)
        
        #sirve para luego asignar un nombre a cada una de las dos secuencias
        #básicamente pone un espacio en blanco y luego si pones 0 (1º sec) o 1 (2º sec) se le asigna el nombre que le des
        #para asignar se utiliza la función append
        sequences = []
        #print(seq)
        sequences.append(seq)
        mus = sequences[0]
        #ratus = sequences[-1]
        #print (mus.seq)
        #print (ratus.seq)
        
        #aplicamos la función en nuestra secuencia de interés
        targetsall = findtargets(mus)
        #targetsall = findtargets (ratus)

Lo siguiente que haremos será ordenar todos los archivos: meteremos los kmers de una especie en una carpeta y los de la otra especie en otra carpeta.

In [ ]:
#primero especificamos el path del que partimos, el path final de la primera especie y el path final de la segunda especie.
generalpath = "C:/Users/nerit/Desktop/mouse and rat genes/kmers/"
rat_path = 'C:/Users/nerit/Desktop/mouse and rat genes/Rat/kmers/'
mouse_path = 'C:/Users/nerit/Desktop/mouse and rat genes/Mouse/kmers/'

#creamos un bucle para la primera especie.
#primero pone como condición que cierta palabra (la especie) esté en el título del archivo
#mueve el archivo con la especie en el nombre a la carpeta de esa especie
#repetimos el bucle para la segunda especie

for k in os.listdir(generalpath):
    #print(k)
    if "Mus" in k:
        mouse = k
        #print(mouse)
        shutil.move(generalpath + k, mouse_path)

for k in os.listdir(generalpath):        
    if "Rattus" in k:
        rat = k
        #print(rat)
        shutil.move(generalpath + k, rat_path) 

Antes de continuar, revisaremos manualmente que no haya casos raros: podría suceder que para un gen no haya posibilidad de diseñar sondas que cumplan todos los requisitos, o que tengamos varias isoformas de un gen para una especie. En esos casos lo que haremos será , manualmente, sacar las excepciones de las carpetas (tanto en mouse cómo en rat) y meterlos en una carpeta nueva (strange genes). 

## Renombrar los archivos

Este paso es muy importante, para que los archivos tengan el nombre "común" del gen en ambas especies. De esta manera podremos trabajar posteriormente con ellos a la vez. Para no tener problemas luego a la hora de hacer todos los cambios es importante que en el nombre no haya ninguna "/" ni ningún paréntesis mas que el que rodea el nombre común del gen.

In [ ]:
#lo primero que haremos será indicar el path de cada especie de forma separada.
#el path será una lista de archivos (usando la función os.chdir)
ratpath = os.chdir('C:/Users/nerit/Desktop/mouse and rat genes/Rat/kmers')
mousepath = os.chdir ('C:/Users/nerit/Desktop/mouse and rat genes/Mouse/kmers')

#aplicamos en cada especie la función para cambiar el nombre que hemos definido anteriormente
rename_rat = change_name(ratpath)
rename_mouse = change_name(mousepath)

## Búsqueda de kmers comunes en ambas especies

El siguiente paso es hacer una búsqueda de aquellos kmers que son comunes en ambas especies, porque esos serán los que utilicemos. El truco de este paso es que no hace falta que los kmers sean idénticos: sabemos que un kmer con hasta 5 mismatches es capaz de hibridar con el RNA. Así pues, decidimos que la manera de buscar los kmers comunes en dos especies iba a ser la siguiente: haríamos una comparación de los kmers posibles de la especie en la que tenemos mayor interés con los kmers de la segunda, añadiéndole a esta hasta 5 mismatches. Después, a la especie de interés le asignaremos los kmers "originales" y a la segunda especie los kmers comunes a esos "originales" permitiendo que tengan hasta 5 mismatches.

In [ ]:
#comenzamos definiendo el path en el que se encuentran los kmers "originales" de cada especie
rat_path = 'C:/Users/nerit/Desktop/mouse and rat genes/Rat/kmers/'
mouse_path = 'C:/Users/nerit/Desktop/mouse and rat genes/Mouse/kmers/'

#definimos un bucle que nos permita leer los archivos que contienen los kmers
for f in os.listdir(rat_path):
    #print(f)
    
    #le indicamos al bucle que los archivos que lee son CSV
    #uno será el que contenga los kmers de la especie 1
    #otro contendrá los kmers de la especie 2
    df1 = pd.read_csv(rat_path+f)
    #print(df1)
    df2 = pd.read_csv(mouse_path+f)
    #print (df2)
    
    #asignamos un espacio vacío a cada especie por cada CSV añadido
    #cuando encuentre kmers coincidentes se asignarán a ese espacio vacío
    hitsrat=[]
    hitsmouse=[]
    
    #dentro del bucle inicial le indicamos que vaya analizando una a una las secuencias de los kmers (en la columna "sequence")
    #cogerá como referencia los kmers de la especie de mayor interés
    for seqmouse in df2['Sequence']:
        #print (seqmouse)
        
        #creamos un bucle interno, que comparará cada kmer de la especie de interés con los kmers de la segunda especie
        #la función regex.findall contará como hit positivo todos los kmers que coincidan con el de la especie de interés
        #al especificar s<=5 le decimos que aceptamos hasta 5 mismatches en el hit
        for seqrat in df1['Sequence']:
            #print (seqrat)
            hit = regex.findall(r"("+seqmouse+"){s<=5}",seqrat)
            
            #en esta condición indicamos que si hay un hit, añadamos a la especie de interés el kmer "original"
            #a la secundaria le añadiremos el hit coincidente, tenga mismatches o no
            #para eso usamos la función .append asociada al espacio vacío que habíamos creado previamente para cada especie
            if hit:
                #print (f)
                #print (seqrat)
                #print (hit)
                hitsrat.append(hit)
                hitsmouse.append(seqmouse)
    
    #ordenamos todos los hits obtenidos en un dataframe (uno por especie) 
    mouse_filtered = pd.DataFrame(data=hitsmouse, columns=['Sequence'])
    rat_filtered = pd.DataFrame(data=hitsrat, columns=['Sequence'])
    #print (mouse_filtered)
    
    #convertimos el dataframe en csv
    #simplemente tendremos que añadir el path donde queremos guardarlo, poniendo {f} para que añada el nombre "común" del gen
    outputname_rat = f'C:/Users/nerit/Desktop/mouse and rat genes/Rat/kmers_filtered/{f}'
    rat_filtered.to_csv(outputname_rat)
    outputname_mouse = f'C:/Users/nerit/Desktop/mouse and rat genes/Mouse/kmers_filtered/{f}'
    mouse_filtered.to_csv(outputname_mouse)

## Fusión de CSVs

En el CSV obtenido sólo tenemos las secuencias de los kmers, sin su posición. Sin embargo, para hacer la selección de los kmers necesitamos esa posición en el mRNA. Para eso haremos una fusión entre el CSV que contiene los kmers "originales" y el CSV que hemos conseguido con los kmers "filtrados" que son comunes a las dos especies. 

In [ ]:
#comenzamos indicando el path de los kmers "originales" (kmer) y "filtrados" (filtered) para ambas especies
kmer_mouse = 'C:/Users/nerit/Desktop/mouse and rat genes/Mouse/kmers/'
filteredkmer_mouse = 'C:/Users/nerit/Desktop/mouse and rat genes/Mouse/kmers_filtered/'
kmer_rat = 'C:/Users/nerit/Desktop/mouse and rat genes/Rat/kmers/'
filteredkmer_rat ='C:/Users/nerit/Desktop/mouse and rat genes/Rat/kmers_filtered/'

#lo haremos para cada especie por separado:
#creamos un bucle que lea los archivos de una especie de uno en uno
    #indicamos que vamos a trabajar con dos dataframes (función "pd.read_csv")
        #el primero es el que contiene los kmers "originales"
        #el segundo es el que contiene los kmers "filtrados"
    #usamos la función "pd.merge" para unir dos csv manteniendo sólo aquello común a ambos
        #indicaremos los dos dataframes que queremos fusionar
        #on -> indicamos qué columna es en la que queremos buscar la coincidencia
    #creamos el nuevo csv que contendrá los kmers ya filtrados con su posición en el mRNA

for f in os.listdir(kmer_mouse):
    #print(f)
    df1 = pd.read_csv(kmer_mouse+f)
    #print(df1)
    df2 = pd.read_csv(filteredkmer_mouse+f)
    #print (df2)
    mergedStuff = pd.merge(df1, df2, on=['Sequence'], how='inner')
    outputname_mouse = f'C:/Users/nerit/Desktop/mouse and rat genes/Mouse/kmers_fused/m{f}'
    mergedStuff.to_csv(outputname_mouse)
    
for f in os.listdir(kmer_rat):
    #print(f)
    df1 = pd.read_csv(kmer_rat+f)
    #print(df1)
    df2 = pd.read_csv(filteredkmer_rat+f)
    #print (df2)
    mergedStuff = pd.merge(df1, df2, on=['Sequence'], how='inner')
    outputname_rat = f'C:/Users/nerit/Desktop/mouse and rat genes/Rat/kmers_fused/r{f}'
    mergedStuff.to_csv(outputname_rat)
